In [244]:
from urllib.request import urlopen, Request

launch_list = []

reg_url_list = ['https://spaceflightnow.com/launch-log-2004-2008/', 
               'https://spaceflightnow.com/launch-log-2009-2011/',
               'https://spaceflightnow.com/launch-log-2012-2014/',
               'https://spaceflightnow.com/launch-log-2015-2016/',
                'https://spaceflightnow.com/launch-log-2017-2018/',
               'https://spaceflightnow.com/launch-log/']

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
for reg_url in reg_url_list: 
    req = Request(url=reg_url, headers=headers) 
    html = urlopen(req).read() 
    html = html.decode('utf-8')
    # print(html) 

    html_str = str(html)
    html_str = html_str[html_str.index('<div class="entry-content clearfix">') + 1:]
    html_str = html_str[html_str.index('<div class="datename">'):]
    html_str = html_str[:html_str.index('</article>'):]
    html_list = []

    while ('<div class="datename">' in html_str):
        html_str = html_str[html_str.index('<div class="datename">') + len('<div class="datename">'):]
        if '<div class="datename">' in html_str:
            html_list.append(html_str[:html_str.index('<div class="datename">')])
        else:
            html_list.append(html_str)
    def getInfo(item, tag1, tag2):
        return item[item.index(tag1) + len(tag1): item.index(tag1) + item[item.index(tag1):].index(tag2)]

    for i, item in enumerate(html_list):
        if 'China' in item:
    #         print(item)
            item = item.replace('&amp;','&')
            item = item.replace('&#8217;',"'")
            item = item.replace('\\xc2\\xa0', " ")
            date = getInfo(item, '<span class="launchdate">', '</span>').strip()
            date = date.replace('Sept.', 'Sep.')
            if ('<a name="jul04"></a>' in date):
                date = date[len('<a name="jul04"></a>'):]
            if '/' in date:
                if date[date.index('/') + 1].isdigit():
                    date = date[:date.index(' ') + 1] + date[date.index('/') + 1:]
                else: # Nov. 30/Dec. 1, 2008
                    date = date[date.index('/') + 1:]
            if ', ' not in date and date.count('.') < 2:
                print('!!!Warning incorrect date form: ' + date)
                date += ', 2021'
                print(date + ' <= changed to')
    #         if date.count('.') == 2:
    #             print('!!!Warning incorrect date form: ' + date)
    #             secondDot = date.rfind('.')
    #             date = date[:secondDot] + ',' + date[secondDot + 1:]
    #             print(date + ' <= changed to')
            if date == 'Jan. 7. 2020':
                date = 'Jan. 7, 2020'
                print('Jan. 7, 2020 changed from Jan. 7. 2020')
            if '.' in date:
                date = datetime.datetime.strptime(date,'%b. %d, %Y').strftime('%Y-%m-%d')
            else:
                date = datetime.datetime.strptime(date,'%B %d, %Y').strftime('%Y-%m-%d')
            try:
                time = getInfo(item, 'Launch time:</span>', '(').strip()#'<span class="strong">')
                time = time[0:2] + ":" + time[2:4]
            except:
                time = ''
                print('time format wrong, cannot read')
            mission = getInfo(item, '<span class="mission">', '</span>').strip()
            if '•' in mission:
                rocket = mission[:mission.index('•')].strip()
                payload = mission[mission.index('•') + len('•'):].strip()
            else:
                rocket = mission.strip()
                payload = ''.strip()
            launchsite = getInfo(item, 'Launch site:</span>', '</div>').strip()
            description = getInfo(item, '<div class="missdescrip">', '</div>').strip().replace('\\xe2\\x80\\x99s', "'")
            description = description.replace('\\n','')
            if 'Read our' in description:
                description = description[: description.index('Read our')] + description[description.lower().rfind('story</a>')  + len('story</a>.'):]
            if description[-1] == ']':
                description = description[: description.rfind('[')].strip()
            launch_list.append({
                'date': date,
                'time': time,
                'rocket': rocket,
                'payload': payload,
                'launchsite': launchsite,
                'description': description,
                'tweetID': -1
            })

time format wrong, cannot read
!!!Warning incorrect date form: July 9
July 9, 2021 <= changed to
!!!Warning incorrect date form: July 6
July 6, 2021 <= changed to
Jan. 7, 2020 changed from Jan. 7. 2020


In [248]:
launch_list

[{'date': '2008-12-23',
  'time': '00:54',
  'rocket': 'Long March 3A',
  'payload': 'Feng Yun 2E',
  'launchsite': 'Xichang, China',
  'description': 'The Long March 3A rocket launched the Feng Yun 2E spacecraft, which will join a fleet of geostationary weather satellites operated by the China Meteorological Administration.',
  'tweetID': -1},
 {'date': '2008-12-15',
  'time': '03:22',
  'rocket': 'Long March 4B',
  'payload': 'Yaogan 5',
  'launchsite': 'Taiyuan, China',
  'description': "A Chinese Long March 4B rocket launched the country's Yaogan 5 satellite for Earth observations.",
  'tweetID': -1},
 {'date': '2008-12-01',
  'time': '04:42',
  'rocket': 'Long March 4B',
  'payload': 'Yaogan 4',
  'launchsite': 'Jiuquan, China',
  'description': "A Chinese Long March 2D rocket launched the country's Yaogan 4 satellite for Earth observations.",
  'tweetID': -1},
 {'date': '2008-11-05',
  'time': '00:15',
  'rocket': 'Long March 2D',
  'payload': 'Shiyan 3 & Chuangxin 1-02',
  'laun

In [249]:
import json

def saveJSON(data, jsonFilename):
    data.sort(key = lambda x: (x['date'], x['time']), reverse = True)
    with open(jsonFilename, 'w') as f:
        json.dump(data , f, ensure_ascii=False)
        
saveJSON(launch_list, '../json/launch_log.json')

In [196]:
'd'.isdigit()

False